In [1]:
import numpy as np
from numpy.fft  import fft2, ifft2
from model import layers, activations, networks, utils, optimizers, losses, metrics, paddings
import matplotlib.pyplot as plt
import os
from PIL import Image

In [ ]:
filepath = 'datasets\\catsanddogs\\Petimages\\Cat'
image_rgb = Image.open(filepath + '\\' + os.listdir(filepath)[7])
image_resized = image_rgb.resize((256, 256))
image = np.asarray(image_resized) / 255

plt.imshow(image, cmap='gray')

In [ ]:
kernel = np.random.randint(-1, 1, (64, 3, 5, 5))

In [ ]:
# %%timeit
p = int(kernel.shape[2] / 2)
c_scipy = np.zeros((image.shape[0] - 2 * p, image.shape[0] - 2 * p, kernel.shape[0]))
for k in np.arange(kernel.shape[0]):
    for c in np.arange(image.shape[2]):
        c_scipy[:, :, k] += utils.convolve_scipy(image[:, :, c], kernel[k, c])

I further optimized the code shown here #https://laurentperrinet.github.io/sciblog/posts/2017-09-20-the-fastest-2d-convolution-in-the-world.html to create my own version of fft convolution.
Performance-wise it gets very close to scipys convolution, but has the advantage of being able to convolve higher dimensional Arrays that are used in CNNs

In [ ]:
def convolve_fft(image, kernel): # array shape (h, w, c), kernel shape (k, c, h, w)
    C = np.zeros((*image.shape[:2], kernel.shape[0]))
    kernel_fft = np.moveaxis(fft2(kernel, s=image.shape[:2]), 1, -1)
    image_fft = fft2(image, axes=(0, 1))
    for k in np.arange(kernel.shape[0]):
        C[:, :, k] = np.sum(np.real(ifft2(image_fft * kernel_fft[k], axes=(0, 1))), axis=2)
    p = int(kernel.shape[2] / 2)
    return C[p : -p, p : -p]

In [ ]:
# %%timeit
c_1 = convolve_fft(image, kernel)

In [ ]:
def convolve_fft2(image, kernel): # array shape (h, w, c), kernel shape (h, w, c, k)
    kernel_fft = fft2(kernel, s=image.shape[:2], axes=(0, 1))
    image_fft = np.repeat(np.expand_dims(fft2(image, axes=(0, 1)), -1), kernel.shape[-1], axis=-1)
    image_fft = np.resize(fft2(image, axes=(0, 1)), kernel_fft.shape)
    C = np.sum(np.real(ifft2(image_fft * kernel_fft, axes=(0, 1))), axis=2)
    p = int(kernel.shape[2] / 2)
    return C[p : -p, p : -p]

In [ ]:
kernel_reshaped = np.moveaxis(np.moveaxis(kernel, 0, -1), 0, -2)
kernel_reshaped.shape

In [ ]:
# %%timeit
c_2 = convolve_fft2(image, kernel_reshaped)

direct comparison

In [2]:
a = np.random.randint(0, 255, (5, 5))
a

array([[180, 241,  83, 139, 182],
       [ 79, 132, 176,  57,  13],
       [186, 122,   8, 194,  40],
       [116, 233,  49, 246, 115],
       [176, 156, 150,  18, 244]])

In [3]:
kernel = np.random.randint(0, 10, (2, 2))

In [4]:
import math
p = math.ceil(kernel.shape[0] / 2)
p

1

In [5]:
c = utils.convolve_2d_fft(a, kernel)
c

array([[4688., 3949., 3801., 1983., 3096.],
       [3389., 4237., 3620., 2759., 3130.],
       [1174., 2765., 3268., 2323., 1446.],
       [2610., 3469., 2151., 2260., 3205.],
       [3231., 4001., 3312., 3273., 3565.]])

In [6]:
c[p:, p:]

array([[4237., 3620., 2759., 3130.],
       [2765., 3268., 2323., 1446.],
       [3469., 2151., 2260., 3205.],
       [4001., 3312., 3273., 3565.]])

In [7]:
d = utils.convolve_scipy(a, kernel)
d

array([[4237, 3620, 2759, 3130],
       [2765, 3268, 2323, 1446],
       [3469, 2151, 2260, 3205],
       [4001, 3312, 3273, 3565]])